##### Copyright 2023 The TF-Agents Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ambientes

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/agents/tutorials/2_environments_tutorial">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/2_environments_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/agents/tutorials/2_environments_tutorial.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/agents/tutorials/2_environments_tutorial.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Introdução

O objetivo do Aprendizado por Reforço (RL) é criar agentes que aprendem ao interagir com um ambiente. No cenário padrão de RL, o agente recebe uma observação a cada timestep e escolhe uma ação. A ação é aplicada ao ambiente, e o ambiente retorna uma recompensa e uma nova observação. O agente treina uma política para escolher ações que maximizam a soma de recompensas, também conhecida como retorno.

No TF-Agents, os ambientes podem ser implementados no Python ou no TensorFlow. Os ambientes do Python são geralmente mais fácil de implementar, entender e depurar. Porém, os ambientes do TensorFlow são mais eficientes e permitem a paralelização natural. O fluxo de trabalho mais comum é implementar um ambiente no Python e usar um dos nossos wrappers para convertê-lo automaticamente para o TensorFlow.

Vamos conferir os ambientes do Python primeiro. Os ambientes do TensorFlow seguem uma API bastante parecida.

## Configuração


Se você ainda não instalou o tf-agents ou gym, execute:

In [ ]:
!pip install tf-agents[reverb]


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
import numpy as np

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

## Ambientes do Python

Os ambientes do Python têm um método `step(action) -> next_time_step` que aplica uma ação ao ambiente e retorna as seguintes informações sobre o próximo passo:

1. `observation`: essa é a parte do estado do ambiente que o agente pode observar para escolher as ações dele no próximo passo.
2. `reward`: o agente está aprendendo a maximizar a soma dessas recompensas em vários passos.
3. `step_type`: as interações com o ambiente geralmente fazem parte de uma sequência/episódio, por exemplo, várias jogadas em um jogo de xadrez. step_type pode ser `FIRST`, `MID` ou `LAST`, para indicar se o timestep é o primeiro, intermediário ou último passo de uma sequência.
4. `discount`: isso é um float que representa o peso atribuído à recompensa no próximo timestep em relação à recompensa no timestep atual.

Elas são agrupadas em uma tupla chamada `TimeStep(step_type, reward, discount, observation)`.

A interface que todos os ambientes do Python precisam implementar está em `environments/py_environment.PyEnvironment`. Os principais métodos são:

In [ ]:
class PyEnvironment(object):

  def reset(self):
    """Return initial_time_step."""
    self._current_time_step = self._reset()
    return self._current_time_step

  def step(self, action):
    """Apply action and return new time_step."""
    if self._current_time_step is None:
        return self.reset()
    self._current_time_step = self._step(action)
    return self._current_time_step

  def current_time_step(self):
    return self._current_time_step

  def time_step_spec(self):
    """Return time_step_spec."""

  @abc.abstractmethod
  def observation_spec(self):
    """Return observation_spec."""

  @abc.abstractmethod
  def action_spec(self):
    """Return action_spec."""

  @abc.abstractmethod
  def _reset(self):
    """Return initial_time_step."""

  @abc.abstractmethod
  def _step(self, action):
    """Apply action and return new time_step."""

Além do método `step()`, os ambientes também fornecem um método `reset()` que inicia uma nova sequência e oferece um `TimeStep` inicial. Não é necessário chamar o método `reset` explicitamente. Presumimos que os ambientes são redefinidos de maneira automática, seja quando chegam ao final de um episódio ou quando step() é chamado pela primeira vez.

Observe que as subclasses não implementam `step()` ou `reset()` diretamente. Em vez disso, elas substituem os métodos `_step()` e `_reset()`. Os timesteps retornados por esses métodos serão armazenados em cache e expostos através de `current_time_step()`.

Os métodos `observation_spec` e `action_spec` retornam um ninho de `(Bounded)ArraySpecs`, que descreve o nome, o formato, o tipo de dados e os intervalos das observações e das ações, respectivamente.

No TF-Agents, nos referimos várias vezes aos ninhos, que são definidos como qualquer estrutura semelhante a uma árvore composta de listas, tuplas, tuplas nomeadas ou dicionários. Eles podem ser compostos arbitrariamente para manter a estrutura das observações e das ações. Isso é muito útil para ambientes mais complexos com várias observações e ações.

### Usando ambientes padrão

O TF Agents tem wrappers integrados para vários ambientes padrão, como OpenAI Gym, DeepMind-control e Atari, para que sigam nossa interface `py_environment.PyEnvironment`. Esses ambientes com wrappers podem ser facilmente carregados usando nossas suítes de ambiente. Vamos carregar o ambiente CartPole no OpenAI Gym e conferir a ação e o time_step_spec.

In [ ]:
environment = suite_gym.load('CartPole-v0')
print('action_spec:', environment.action_spec())
print('time_step_spec.observation:', environment.time_step_spec().observation)
print('time_step_spec.step_type:', environment.time_step_spec().step_type)
print('time_step_spec.discount:', environment.time_step_spec().discount)
print('time_step_spec.reward:', environment.time_step_spec().reward)


Podemos ver que o ambiente espera ações do tipo `int64` em [0, 1] e retorna `TimeSteps` em que as observações são um vetor `float32` de comprimento 4 e o fator de desconto é um `float32` em [0.0, 1.0]. Agora, vamos tentar usar uma ação fixa `(1,)` para um episódio inteiro.

In [ ]:
action = np.array(1, dtype=np.int32)
time_step = environment.reset()
print(time_step)
while not time_step.is_last():
  time_step = environment.step(action)
  print(time_step)

### Criando seu próprio ambiente do Python

Para vários clientes, um caso de uso comum é aplicar um dos agentes padrão (veja agents/) no TF-Agents ao problema deles. Para isso, eles precisam enquadrar o problema como um ambiente. Confira como implementar um ambiente no Python.

Digamos que queremos treinar um agente para jogar o seguinte jogo de cartas (inspirado no Blackjack):

1. Para jogar, é usado um baralho infinito de cartas numeradas de 1...10.
2. Durante cada vez, o agente pode fazer 2 coisas: receber uma nova carta aleatória ou interromper a rodada atual.
3. O objetivo é que a soma das suas cartas chegue o mais próximo possível de 21 no final da rodada, sem passar disso.

Um ambiente que representa o jogo pode parecer com isto:

1. Ações: temos 2 ações. Ação 0: receber uma nova carta. Ação 1: terminar a rodada atual.
2. Observações: a soma das cartas na rodada atual.
3. Recompensa: o objetivo é chegar o mais perto possível de 21 sem ultrapassar disso. Portanto, podemos obter isso usando a seguinte recompensa no final da rodada: soma_das_cartas - 21 if soma_das_cartas &lt;= 21, else -21


In [ ]:
class CardGameEnv(py_environment.PyEnvironment):

  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int32, minimum=0, maximum=1, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(1,), dtype=np.int32, minimum=0, name='observation')
    self._state = 0
    self._episode_ended = False

  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def _reset(self):
    self._state = 0
    self._episode_ended = False
    return ts.restart(np.array([self._state], dtype=np.int32))

  def _step(self, action):

    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()

    # Make sure episodes don't go on forever.
    if action == 1:
      self._episode_ended = True
    elif action == 0:
      new_card = np.random.randint(1, 11)
      self._state += new_card
    else:
      raise ValueError('`action` should be 0 or 1.')

    if self._episode_ended or self._state >= 21:
      reward = self._state - 21 if self._state <= 21 else -21
      return ts.termination(np.array([self._state], dtype=np.int32), reward)
    else:
      return ts.transition(
          np.array([self._state], dtype=np.int32), reward=0.0, discount=1.0)

Vamos verificar se fizemos tudo certo definindo o ambiente acima. Ao criar seu próprio ambiente, você precisa conferir se as observações e os time_steps gerados seguem os formatos e tipos corretos conforme definido nas suas especificações. Eles são usados para gerar o grafo do TensorFlow e, portanto, podem criar problemas difíceis de depurar se estiverem errados.

Para validar seu ambiente, usaremos uma política aleatória para gerar ações, e vamos iterar mais de 5 episódios para garantir que tudo esteja funcionando como esperado. Um erro será gerado se recebermos um time_step que não seguir as especificações do ambiente.

In [ ]:
environment = CardGameEnv()
utils.validate_py_environment(environment, episodes=5)

Agora que sabemos que o ambiente está funcionando como esperado, vamos executar esse ambiente usando uma política fixa: peça 3 cartas e encerre a rodada.

In [ ]:
get_new_card_action = np.array(0, dtype=np.int32)
end_round_action = np.array(1, dtype=np.int32)

environment = CardGameEnv()
time_step = environment.reset()
print(time_step)
cumulative_reward = time_step.reward

for _ in range(3):
  time_step = environment.step(get_new_card_action)
  print(time_step)
  cumulative_reward += time_step.reward

time_step = environment.step(end_round_action)
print(time_step)
cumulative_reward += time_step.reward
print('Final Reward = ', cumulative_reward)

### Wrappers de ambiente

Um wrapper de ambiente aceita um ambiente do Python e retorna uma versão modificada do ambiente. Ambos os ambientes original e modificado são instâncias de `py_environment.PyEnvironment`, e vários wrappers podem ser ligados juntos em cadeia.

Alguns wrappers comuns podem ser encontrados em `environments/wrappers.py`. Por exemplo:

1. `ActionDiscretizeWrapper`: converte um espaço de ação contínuo para um espaço de ação discreto.
2. `RunStats`: captura as estatísticas de execução do ambiente, como número de passos tomados, número de episódios concluídos etc.
3. `TimeLimit`: termina o episódio após um número fixo de passos.


#### Exemplo 1: Action Discretize Wrapper

InvertedPendulum é um ambiente do PyBullet que aceita ações contínuas no intervalo `[-2, 2]`. Se quisermos treinar um agente de ação discreta como DQN nesse ambiente, precisamos discretizar (quantizar) o espaço da ação. Isso é exatamente o que `ActionDiscretizeWrapper` faz. Compare a `action_spec` antes e depois do wrapping:

In [ ]:
env = suite_gym.load('Pendulum-v1')
print('Action Spec:', env.action_spec())

discrete_action_env = wrappers.ActionDiscretizeWrapper(env, num_actions=5)
print('Discretized Action Spec:', discrete_action_env.action_spec())

O `discrete_action_env` envolvido é uma instância de `py_environment.PyEnvironment` e pode ser tratado como um ambiente Python normal.


## Ambientes do TensorFlow

A interface para os ambientes do TF é definida em `environments/tf_environment.TFEnvironment` e se parece muito com os ambientes do Python. Os ambientes do TF diferem dos ambientes do Python de algumas maneiras:

- Eles geram objetos de tensores em vez de arrays
- Os ambientes do TF adicionam uma dimensão de lote aos tensores gerados em comparação com as especificações.

A conversão dos ambientes do Python em TFEnvs permite que o TensorFlow paralelize as operações. Por exemplo, é possível definir uma `collect_experience_op` que coleta dados do ambiente e adiciona a um `replay_buffer`, e uma `train_op` que lê `replay_buffer` e treina o agente, e executá-las naturalmente em paralelo no TensorFlow.

In [ ]:
class TFEnvironment(object):

  def time_step_spec(self):
    """Describes the `TimeStep` tensors returned by `step()`."""

  def observation_spec(self):
    """Defines the `TensorSpec` of observations provided by the environment."""

  def action_spec(self):
    """Describes the TensorSpecs of the action expected by `step(action)`."""

  def reset(self):
    """Returns the current `TimeStep` after resetting the Environment."""
    return self._reset()

  def current_time_step(self):
    """Returns the current `TimeStep`."""
    return self._current_time_step()

  def step(self, action):
    """Applies the action and returns the new `TimeStep`."""
    return self._step(action)

  @abc.abstractmethod
  def _reset(self):
    """Returns the current `TimeStep` after resetting the Environment."""

  @abc.abstractmethod
  def _current_time_step(self):
    """Returns the current `TimeStep`."""

  @abc.abstractmethod
  def _step(self, action):
    """Applies the action and returns the new `TimeStep`."""

O método `current_time_step()` retorna o time_step atual e inicializa o ambiente, se necessário.

O método `reset()` força a redefinição do ambiente e retorna o current_step.

Se a `action` não depender do `time_step` anterior, será necessária uma `tf.control_dependency` no modo `Graph`.

Por enquanto, vamos ver como `TFEnvironments` são criados.

### Criando seu próprio ambiente do TensorFlow

Isso é mais complicado do que criar ambientes no Python, então não vamos abordar aqui neste colab. Um exemplo está disponível [aqui](https://github.com/tensorflow/agents/blob/master/tf_agents/environments/tf_environment_test.py). O caso de uso mais comum é implementar seu ambiente no Python e adicionar nosso wrapper `TFPyEnvironment` a ele no TensorFlow (veja abaixo).

### Envolvendo um ambiente do Python no TensorFlow

Podemos envolver facilmente qualquer ambiente do Python em um ambiente do TensorFlow usando o wrapper `TFPyEnvironment`.

In [ ]:
env = suite_gym.load('CartPole-v0')
tf_env = tf_py_environment.TFPyEnvironment(env)

print(isinstance(tf_env, tf_environment.TFEnvironment))
print("TimeStep Specs:", tf_env.time_step_spec())
print("Action Specs:", tf_env.action_spec())

Observe o novo tipo das especificações: `(Bounded)TensorSpec`.

### Exemplos de uso

#### Exemplo simples

In [ ]:
env = suite_gym.load('CartPole-v0')

tf_env = tf_py_environment.TFPyEnvironment(env)
# reset() creates the initial time_step after resetting the environment.
time_step = tf_env.reset()
num_steps = 3
transitions = []
reward = 0
for i in range(num_steps):
  action = tf.constant([i % 2])
  # applies the action and returns the new TimeStep.
  next_time_step = tf_env.step(action)
  transitions.append([time_step, action, next_time_step])
  reward += next_time_step.reward
  time_step = next_time_step

np_transitions = tf.nest.map_structure(lambda x: x.numpy(), transitions)
print('\n'.join(map(str, np_transitions)))
print('Total reward:', reward.numpy())

#### Episódios inteiros

In [ ]:
env = suite_gym.load('CartPole-v0')
tf_env = tf_py_environment.TFPyEnvironment(env)

time_step = tf_env.reset()
rewards = []
steps = []
num_episodes = 5

for _ in range(num_episodes):
  episode_reward = 0
  episode_steps = 0
  while not time_step.is_last():
    action = tf.random.uniform([1], 0, 2, dtype=tf.int32)
    time_step = tf_env.step(action)
    episode_steps += 1
    episode_reward += time_step.reward.numpy()
  rewards.append(episode_reward)
  steps.append(episode_steps)
  time_step = tf_env.reset()

num_steps = np.sum(steps)
avg_length = np.mean(steps)
avg_reward = np.mean(rewards)

print('num_episodes:', num_episodes, 'num_steps:', num_steps)
print('avg_length', avg_length, 'avg_reward:', avg_reward)